In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.misc import imread,imresize,imsave
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D,AveragePooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
from pytictoc import TicToc
from keras.utils import np_utils
from sklearn.utils import shuffle
from keras.layers import BatchNormalization as Batch_Norm
from keras.layers.core import Dense,Flatten,Dropout,Activation

ModuleNotFoundError: No module named 'keras'

In [ ]:
def Bayer_Pattern(img,bayer_pattern):

    """Returns the 2D bayer pattern version of a 3D rgb image
       Input the rgb image and bayer pattern as a list declared in row major format
       bayer_pattern = ['green','blue','red','green']; Declared in row major format"""

      
    dictionary = {'red':0 , 'green':1 , 'blue':2};# red is layer 1;green is layer 2;blue is layer 3

    bayer_mask = np.empty_like(img[:,:,0]);# Returns a 2D array with bayer pattern

    # Constructing Bayer_image
    bayer_mask[0:img.shape[0]:2,0:img.shape[1]:2] = img[0:img.shape[0]:2,0:img.shape[1]:2,dictionary[bayer_pattern[0]]]
    bayer_mask[0:img.shape[0]:2,1:img.shape[1]:2] = img[0:img.shape[0]:2,1:img.shape[1]:2,dictionary[bayer_pattern[1]]]
    bayer_mask[1:img.shape[0]:2,0:img.shape[1]:2] = img[1:img.shape[0]:2,0:img.shape[1]:2,dictionary[bayer_pattern[2]]]
    bayer_mask[1:img.shape[0]:2,1:img.shape[1]:2] = img[1:img.shape[0]:2,1:img.shape[1]:2,dictionary[bayer_pattern[3]]]
    
    return bayer_mask

def demosaic(img,bayer_pattern,method):
    
    """Returns a Demosaiced image for a given bayer img and the bayer pattern used"""
    
    #bayer_pattern = ['green','blue','red','green']
    where = {0:(0,0),1:(0,1),2:(1,0),3:(1,1)}

    red = where[bayer_pattern.index('red')]
    blue = where[bayer_pattern.index('blue')]
    green_1 = where[bayer_pattern.index('green')]
    
    if green_1 == 0:
        green_2 = where[3]
        
    else:
        green_2 = where[2]
    
    if method == 'nearest neighbor':
        interp_red = np.reshape(np.repeat(img[red[0]:img.shape[0]:2,red[1]:img.shape[1]:2]
                                ,4,axis=0)[:,0],(img.shape[0],2))
        interp_blue = np.reshape(np.repeat(img[blue[0]:img.shape[0]:2,blue[1]:img.shape[1]:2]
                                 ,4,axis=0)[:,0],(img.shape[0],2))
        
        interp_green_1 = np.reshape(np.repeat(img[green_1[0]:img.shape[0]:2,green_1[1]:img.shape[1]:2]
                                 ,4,axis=0)[:,0],(img.shape[0],2))
        interp_green_2 = np.reshape(np.repeat(img[green_2[0]:img.shape[0]:2,green_2[1]:img.shape[1]:2]
                                 ,4,axis=0)[:,0],(img.shape[0],2))


        for i in range(1,int(img.shape[1]/2)):
            interp_red = np.append(interp_red,np.reshape(np.repeat(img[red[0]:img.shape[0]:2,red[1]:img.shape[1]:2]
                                    ,4,axis=0)[:,i],(img.shape[0],2)),axis=1)
            interp_blue = np.append(interp_blue,np.reshape(np.repeat(img[blue[0]:img.shape[0]:2,blue[1]:img.shape[1]:2]
                                    ,4,axis=0)[:,i],(img.shape[0],2)),axis=1)
            
            interp_green_1 = np.append(interp_green_1,np.reshape(np.repeat(img[0:img.shape[0]:2,0:img.shape[1]:2]
                                                                           ,4,axis=0)[:,i],(img.shape[0],2)),axis=1)
            interp_green_2 = np.append(interp_green_2,np.reshape(np.repeat(img[1:img.shape[0]:2,1:img.shape[1]:2]
                                                                           ,4,axis=0)[:,i],(img.shape[0],2)),axis=1)
        
        interp_green_1[1:interp_green_1.shape[0]:2,:] = 0
        interp_green_2[0:interp_green_2.shape[0]:2,:] = 0

        interp_img = np.stack((interp_red,interp_blue,interp_green_1+interp_green_2),axis=-1)
    return interp_img  

In [ ]:
def Create_Patch(Read_dir,Write_dir,Num_Patches,sz):
    
    from pytictoc import TicToc
    from scipy.misc import imread,imresize,imsave
    """ Read_dir is the image containing folder
     Write_dir is the Directory where the patches are to be saved
     Num_Patches is the maximum number of patches per Classes
     sz is the Patch size
     """ 
    # Make The Write_Directory
    Split_Folder = Write_dir.split("\\")
    index = 0
    Main_Folder = ""
    while(index <= len(Split_Folder)-1):
        if index == 0:
            Main_Folder = Main_Folder + Split_Folder[index]
        else:
            os.chdir(Main_Folder)
            os.mkdir(Split_Folder[index])
            Main_Folder = Main_Folder + "\\" +Split_Folder[index]
        index+=1

    t = TicToc()
    # See What The Main Folder Contains 
    Subfolders = os.listdir(Read_dir)
    i = 0
    for files in Subfolders:
        Sub_dir = Read_dir + "\\" + Subfolders[i]
        Sub_dir_list = os.listdir(Sub_dir)
        os.chdir(Main_Folder)
        os.mkdir(Subfolders[i])
        os.chdir(Main_Folder+"\\"+Subfolders[i])
        print("Processing Folder", Subfolders[i])
        indicator = 1
        find = False
        j = 0
        for files in Sub_dir_list:
            if Sub_dir_list[j] == "Thumbs.db":
                j+=1
            else:
                FullFileName = Sub_dir + "\\" + Sub_dir_list[j]
                I = imread(FullFileName)
                t.tic()
                print("Processing Image",Sub_dir_list[j])
                k = 1
                row_size,col_size,channels = I.shape
                for r in range(int(row_size/sz)):
                    for c in range(int(col_size/sz)):
                        Write_File_Name = "blk_" + str(k) + "_" + Sub_dir_list[j]
                        imsave(Write_File_Name, I[(r+1)*sz-sz:(r+1)*sz,(c+1)*sz-sz:(c+1)*sz,:])
                        k+=1
                        indicator+=1
                        if Num_Patches!='all':
                            if indicator > Num_Patches:
                                find = True
                                break
                    if find:
                        break
                t.toc()
                if find:
                    break
                j+=1

        i+=1
    return print("Action Completed")

In [ ]:
time = TicToc()
Folder = "G:\\Camera_Model_Data\\"
Sub_Folders = os.listdir(Folder)
x_data = []
y_data = []
bayer_pattern = ['green','blue','red','green']

for i in range(len(Sub_Folders)):
    os.chdir(Folder+Sub_Folders[i])
    print("Processing Folder",Sub_Folders[i])
    for j in range(len(os.listdir())):
        time.tic()
        img = imread(os.listdir()[j])
        bayer_img = Bayer_Pattern(img,bayer_pattern)
        demosaiced_img = demosaic(bayer_img,bayer_pattern,method='nearest neighbor')
        x_data.append(img-demosaiced_img)
        y_data.append(i) 
        time.toc()
        print("Processing image number",j)

x_data = np.array(x_data,dtype='float32')
y_data = np_utils.to_categorical(np.array(y_data))
x_data,y_data = shuffle(x_data,y_data)

In [ ]:
inputs = Input(x_data.shape[1:])

s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(64, (3, 3)) (s)
bn_1 = Batch_Norm()(c1)
relu_1 = Activation('relu')(bn_1)

c2 = Conv2D(32, (3, 3), activation='elu') (relu_1)
bn_2 = Batch_Norm()(c2)
relu_2 = Activation('relu')(bn_2)

p1 = MaxPooling2D((2, 2)) (relu_2)

c3 = Conv2D(32, (3, 3)) (p1)
bn_3 = Batch_Norm()(c3)
relu_3 = Activation('relu')(bn_3)

p2 = AveragePooling2D((5,5)) (relu_3)

flat = Flatten()(p2)

fc_1 = Dense(128, activation='tanh') (flat)
drop_1 = Dropout(0.3) (fc_1)

outputs = Dense(num_classes, activation='softmax') (drop_1)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

# Fit model
earlystopper = EarlyStopping(patience=5)
checkpointer = ModelCheckpoint('G:\\Camera_Model_1.h5', verbose=1)
model.fit(x_data, y_data,validation_split=0.2,batch_size=16, epochs=50,
          callbacks=[earlystopper, checkpointer])
#model.save("E:\\Google Drive\\Segmentation_Competition\\Models\\Model_2.h5")